In [17]:
import pandas as pd
from fastai import *
from fastai.text.all import *
articleDetails = pd.read_pickle('headlines.pkl')

In [18]:
articleDetails.head()

,Link,Headline,Post Time
0,https://order-order.com/2022/06/13/davis-slams-us-legal-system-over-assange/,Davis Slams US Legal System Over Assange,2022-06-13 13:00:00
1,https://order-order.com/2022/06/13/reform-uk-launches-74-billion-emergency-recovery-plan/,Reform UK Launches £74 Billion ‘Emergency Recovery Plan’,2022-06-13 11:40:00
2,https://order-order.com/2022/06/13/watch-starmer-insists-theres-no-problem-here-over-sleaze-probe/,Starmer Insists “There’s No Problem Here” Over Sleaze Probe,2022-06-13 11:19:00
3,https://order-order.com/2022/06/13/judge-rules-cadwalladr-claims-of-russian-backing-for-banks-were-defamatory-awards-no-damages/,"Judge Rules Cadwalladr Claims of Russian Backing for Banks Were Defamatory, Awards No Damages",2022-06-13 10:37:00
4,https://order-order.com/2022/06/13/breaking-starmer-under-investigation-by-commons-sleaze-commissioner/,Starmer Under Investigation by Commons Sleaze Commissioner,2022-06-13 10:20:00


In [19]:
print("We have " + str(len(articleDetails['Headline'])) + " headlines in our Corpus of data")

We have 37772 headlines in our Corpus of data


In [20]:
# Average length of headline
sum_lens = 0
for i in range(0, len(articleDetails['Headline'])):
    sum_lens += len(articleDetails['Headline'][i].split())
    
average_length = (sum_lens / len(articleDetails['Headline']))
print(average_length)

6.851953828232554


In [21]:
# Most Popular first words
def find_in_list_of_tuple(list, target):
    return_value = False
    for j in range(0, len(list)):
        if list[j][0] == target:
            return_value = True
    return return_value

def find_index_in_list_of_tuple(list, target):
    return_value = False
    for j in range(0, len(list)):
        if list[j][0] == target:
            return_value = j
    return return_value


def generateLeadingStringArray(array, num_words):
    local_arr = []
    for i in range(0, len(array)):
        first_word_of_string = ' '.join(array[i].split()[:num_words])
        if find_in_list_of_tuple(local_arr, first_word_of_string) == True:
            # Word has already been added to local_arr
            local_arr[find_index_in_list_of_tuple(local_arr, first_word_of_string)][1] += 1
        else:
            local_arr.append([first_word_of_string, 1])

    local_arr = sorted(local_arr, key=lambda tup: tup[1], reverse=True)
    
    return local_arr

In [22]:
most_popular_first_word = generateLeadingStringArray(articleDetails['Headline'], 1)
most_popular_two_words = generateLeadingStringArray(articleDetails['Headline'], 2)
most_popular_three_words = generateLeadingStringArray(articleDetails['Headline'], 3)

print(most_popular_first_word[:5])
print(most_popular_two_words[:5])
print(most_popular_three_words[:5])

[['Labour', 1037], ['Saturday', 646], ['Friday', 644], ['WATCH:', 572], ['The', 543]]
[['Friday Caption', 638], ['Rich’s Monday', 423], ['Saturday Seven', 413], ['Rich &', 284], ['Saturday 7-Up', 209]]
[['Friday Caption Contest', 509], ['Rich’s Monday Morning', 423], ['Saturday Seven Up', 413], ['Saturday 7-Up', 209], ['+ + +', 167]]


In [23]:
dls = TextDataLoaders.from_df(articleDetails, path="./", text_col='Headline', valid_pct=0.3, is_lm=True)

In [24]:
model = language_model_learner(dls, AWD_LSTM)

In [25]:
model.fit_one_cycle(5)

epoch,train_loss,valid_loss,time
0,5.522987,4.897814,29:58
1,4.861944,4.165838,29:54
2,4.354949,3.849818,30:02
3,4.102753,3.750346,29:54
4,4.025673,3.737153,29:57


In [29]:
model.export()

In [30]:
model = load_learner('export.pkl')

In [31]:
def name_of_obj(obj, namespace):
    return [name for name in namespace if namespace[name] is obj]

def getGeneration(leading_string_arr, number_of_words=10, min_length=1, max_length=3, number_of_generations=3):
    model_output = ""
    model_output = model_output + "----------------\n"
    model_output = model_output + "Using the array: " + name_of_obj(leading_string_arr, globals())[0] + "\n"
    for i in range(0, number_of_words):
        if leading_string_arr[i][0][-1] == ":":
            model_output = model_output + leading_string_arr[i][0] + "\n"
        else:
            model_output = model_output + leading_string_arr[i][0] + ":\n"
        for j in range(min_length, max_length):
            num_words = int(average_length*(j+1))
            model_output = model_output + "\t" + "With length of " + str(num_words) + ":\n"
            for k in range(0, number_of_generations):
                with model.no_bar(), model.no_logging():
                    model_output = model_output + "\t\t" + model.predict(leading_string_arr[i][0], n_words=num_words) + "\n"
            
    print(model_output)

In [32]:
#getGeneration(most_popular_first_word)
getGeneration(most_popular_two_words)
#getGeneration(most_popular_three_words)

----------------
Using the array: most_popular_first_word
Labour:
	With length of 13:
		Labour : The Concession Edition Boat Race :
		Labour Strife should fight for the supremacy of Heat Allies ?
		Labour Douglas Sir Campaigned Richard Hack
	With length of 20:
		Labour : Who Wanted Danny SNEAK ? New People # socialists launching music !
		Labour Gerard : An Rakes Tribute from Mass Governance to the British Nation
		Labour Career Best Expenses de Training Corona : George WATCH Treasury Guto
Saturday:
	With length of 13:
		Saturday Seven Up Len Takes a Day
		Saturday Seven Up dem Part a Country on Sunday
		Saturday Seven Up : Yes alliance Sunday Mail
	With length of 20:
		Saturday of the Super Morning Repeat CATCH SEAL View Case for
		Saturday Seven Up Saturday Seven Up Left Hand Looks Daily
		Saturday Seven Up despite mass launch , Saturday Seven Up lurking Guy Fawkes
Friday:
	With length of 13:
		Friday Friday Memo Friday Caption Contest
		Friday Caption Contest ( 2008 Friday Caption C